In [1]:
import torch
import rocket as rk
from rocket.llg import structurefactors as llg_sf
import numpy as np

In [2]:
tng_file = "../../run_openfold/test_cases/6lzm/6lzm-tng_withrfree.mtz"
tng = rk.utils.load_mtz(tng_file)

In [3]:
true_pdb = "../../run_openfold/test_cases/6lzm/6lzm_noalts.pdb"

In [4]:
# Ensures the same HKLs are kept as from PhaserTNG
eps = torch.tensor(tng["EPS"].values, device=rk.utils.try_gpu())
bin_labels = torch.tensor(tng["BIN"].values, device=rk.utils.try_gpu())
sfc_true = llg_sf.initial_SFC(
        true_pdb, tng_file, "FP", "SIGFP", Freelabel="R-free-flags", device="cuda:0"
    )

amps_true = torch.tensor(tng["FEFF"].values, device=rk.utils.try_gpu())

sigmaN_true = llg_sf.calculate_Sigma_atoms(amps_true, eps, bin_labels)
print(sigmaN_true)

E_true = llg_sf.normalize_Fs(
    amps_true,
    eps,
    sigmaN_true,
    bin_labels,
)

Fs = getattr(sfc_true,  "Ftotal_HKL")
phitrue = rk.utils.assert_numpy(
    llg_sf.ftotal_phis(Fs, np.array(tng["dHKL"]))
)


tensor([1135837.6250,  947530.2500,  839351.8125, 1193766.2500, 1521719.5000,
        1332498.3750,  977166.1250,  728949.8750,  566573.0625,  382227.2812,
         330058.7188,  258629.7656,  189224.2031,  179856.7969,  161263.8438,
         138684.0000,  135730.1875,  124165.9844,  107336.3672,   91685.8828,
          73555.0000,   65642.3359,   55153.4492,   44488.5391,   41224.5078,
          40820.4336,   31143.1152,   28760.1875], device='cuda:0')


In [5]:
#np.save("../../run_openfold/test_cases/6lzm/6lzm_allchains-Etrue-solventTrue.npy", rk.utils.assert_numpy(E_true))
#np.save("../../run_openfold/test_cases/6lzm/6lzm_allchains-phitrue-solventTrue.npy", phitrue)